## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


## 0) Prepare booking Sample (we might need this for later)

In [1]:
!pwd

/home/dsc/Data/Chall_Python


In [2]:
cd ../challenge/

/home/dsc/Data/challenge


In [3]:
ls -l

total 1014264
-rw-rw-r--. 1 dsc dsc 554970628 Jan  4  2016 bookings.csv.bz2
-rw-rw-r--. 1 dsc dsc    425006 Nov 30 13:59 bookings.sample.csv
-rw-rw-r--. 1 dsc dsc 483188920 Jan  4  2016 searches.csv.bz2


In [5]:
! bzcat bookings.csv.bz2 | head -10000 > bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings.csv.bz2, output file = (stdout)


In [4]:
! wc -l  bookings.sample.csv

1000 bookings.sample.csv


In [5]:
! bzip2 bookings.sample.csv

In [6]:
! bzcat bookings.sample.csv.bz2 | wc -l

1000


## 1) Get familiar with data

In [7]:
import pandas as pd

### What if we dont want to read the whole file?

In [8]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^')

In [10]:
b=pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)

In [11]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 38 columns):
act_date               1000 non-null object
source                 1000 non-null object
pos_ctry               1000 non-null object
pos_iata               1000 non-null object
pos_oid                1000 non-null object
rloc                   1000 non-null object
cre_date               1000 non-null object
duration               1000 non-null int64
distance               1000 non-null int64
dep_port               1000 non-null object
dep_city               1000 non-null object
dep_ctry               1000 non-null object
arr_port               1000 non-null object
arr_city               1000 non-null object
arr_ctry               1000 non-null object
lst_port               1000 non-null object
lst_city               1000 non-null object
lst_ctry               1000 non-null object
brd_port               1000 non-null object
brd_city               1000 non-null object
brd_ctry        

In [13]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,...,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [18]:
b.columns

Index([u'act_date           ', u'source', u'pos_ctry', u'pos_iata',
       u'pos_oid  ', u'rloc          ', u'cre_date           ', u'duration',
       u'distance', u'dep_port', u'dep_city', u'dep_ctry', u'arr_port',
       u'arr_city', u'arr_ctry', u'lst_port', u'lst_city', u'lst_ctry',
       u'brd_port', u'brd_city', u'brd_ctry', u'off_port', u'off_city',
       u'off_ctry', u'mkt_port', u'mkt_city', u'mkt_ctry', u'intl',
       u'route          ', u'carrier', u'bkg_class', u'cab_class',
       u'brd_time           ', u'off_time           ', u'pax', u'year',
       u'month', u'oid      '],
      dtype='object')

In [19]:
b.columns.tolist()

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [17]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [87]:
b.columns.str.strip()
#b.columns=b.columns.str.strip()

In [20]:
b.shape

(1000, 38)

In [22]:
description=b.describe()

In [28]:
type (description)

pandas.core.frame.DataFrame

In [29]:
description

,duration,distance,intl,pax,year,month
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.0
mean,23935.433000,407.569000,0.643000,0.532000,2013.0,3.0
std,54459.101785,1490.087967,0.479355,1.976077,0.0,0.0
min,61.000000,0.000000,0.000000,-8.000000,2013.0,3.0
25%,3493.000000,0.000000,0.000000,-1.000000,2013.0,3.0
50%,6891.000000,0.000000,1.000000,1.000000,2013.0,3.0
75%,17940.000000,0.000000,1.000000,1.000000,2013.0,3.0
max,482174.000000,10995.000000,1.000000,21.000000,2013.0,3.0


In [35]:
b.describe()['distance']

count     1000.000000
mean       407.569000
std       1490.087967
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      10995.000000
Name: distance, dtype: float64

In [36]:
b.describe(include='all')

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
count,1000,1000,1000,1000,1000,1000,1000,1000.000000,1000.000000,1000,...,1000,1000,1000,1000,1000,1000,1000.000000,1000.0,1000.0,1000
unique,31,5,53,278,281,352,57,NaN,NaN,176,...,634,86,26,3,688,704,NaN,NaN,NaN,45
top,2013-03-06 00:00:00,1P,US,83ccf07a55606976f78e63fcc805f58c,1490c3c5f95f04fa076fe15c775a80a9,a37584d1485cb35991e4ff1a2ba92262,2013-03-04 00:00:00,NaN,NaN,LAX,...,OSLISTISU,FK,V,Y,2013-03-10 18:20:00,2013-03-11 03:52:53,NaN,NaN,NaN,NULL
freq,75,372,402,59,59,12,84,NaN,NaN,30,...,6,122,80,897,6,6,NaN,NaN,NaN,861
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23935.433000,407.569000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.532000,2013.0,3.0,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54459.101785,1490.087967,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.976077,0.0,0.0,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.000000,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-8.000000,2013.0,3.0,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3493.000000,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,2013.0,3.0,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6891.000000,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013.0,3.0,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17940.000000,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,2013.0,3.0,NaN


In [38]:
b.isnull()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
b.isnull().sum()

act_date               0
source                 0
pos_ctry               0
pos_iata               0
pos_oid                0
rloc                   0
cre_date               0
duration               0
distance               0
dep_port               0
dep_city               0
dep_ctry               0
arr_port               0
arr_city               0
arr_ctry               0
lst_port               0
lst_city               0
lst_ctry               0
brd_port               0
brd_city               0
brd_ctry               0
off_port               0
off_city               0
off_ctry               0
mkt_port               0
mkt_city               0
mkt_ctry               0
intl                   0
route                  0
carrier                0
bkg_class              0
cab_class              0
brd_time               0
off_time               0
pax                    0
year                   0
month                  0
oid                    0
dtype: int64

In [43]:
non_null_counts = b.count()

In [44]:
non_null_counts

act_date               1000
source                 1000
pos_ctry               1000
pos_iata               1000
pos_oid                1000
rloc                   1000
cre_date               1000
duration               1000
distance               1000
dep_port               1000
dep_city               1000
dep_ctry               1000
arr_port               1000
arr_city               1000
arr_ctry               1000
lst_port               1000
lst_city               1000
lst_ctry               1000
brd_port               1000
brd_city               1000
brd_ctry               1000
off_port               1000
off_city               1000
off_ctry               1000
mkt_port               1000
mkt_city               1000
mkt_ctry               1000
intl                   1000
route                  1000
carrier                1000
bkg_class              1000
cab_class              1000
brd_time               1000
off_time               1000
pax                    1000
year                

In [40]:
#pd.set_option('display.max_columns', 100)
pd.set_option('display.max_columns', None)


In [41]:
b.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


## 2) Select the columns of interest 

In [45]:
b = pd.read_csv('bookings.csv.bz2',sep='^',usecols=['arr_port','pax', 'year'], nrows=1000)

In [47]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 3) What to do with NaN?



In [48]:
b.isnull().sum()

arr_port    0
pax         0
year        0
dtype: int64

In [49]:
b.count()

arr_port    1000
pax         1000
year        1000
dtype: int64

In [50]:
b=b.dropna()

### What does df.dropna() do?

In [68]:
#np.random.randn
#univariate “normal” (Gaussian) distribution of mean 0 and variance 1
import numpy as np
data=np.random.randn(5, 2)
df = pd.DataFrame(data,columns=['one', 'two' ])
df

,one,two
0,-2.353152,-0.193513
1,-0.506341,0.728012
2,0.297720,0.340959
3,-0.411007,-1.242377
4,-1.658080,-0.262199


In [69]:
df['three']=[None, None, 1, 1, None]
df

,one,two,three
0,-2.353152,-0.193513,NaN
1,-0.506341,0.728012,NaN
2,0.297720,0.340959,1.0
3,-0.411007,-1.242377,1.0
4,-1.658080,-0.262199,NaN


In [79]:
df.dropna()

,one,two,three
2,0.297720,0.340959,1.0
3,-0.411007,-1.242377,1.0


In [80]:
df.dropna(how='all')

,one,two,three
0,-2.353152,-0.193513,NaN
1,-0.506341,0.728012,NaN
2,0.297720,0.340959,1.0
3,-0.411007,-1.242377,1.0
4,-1.658080,-0.262199,NaN


In [78]:
df.dropna(how='any',axis=1)

,one,two
0,-2.353152,-0.193513
1,-0.506341,0.728012
2,0.297720,0.340959
3,-0.411007,-1.242377
4,-1.658080,-0.262199


#### What if we want to drop just based on second column NaN?

In [71]:
df['two'].isnull()

0    False
1    False
2    False
3    False
4    False
Name: two, dtype: bool

In [72]:
~df['two'].isnull()

0    True
1    True
2    True
3    True
4    True
Name: two, dtype: bool

In [73]:
df[~(df['two'].isnull())]

,one,two,three
0,-2.353152,-0.193513,NaN
1,-0.506341,0.728012,NaN
2,0.297720,0.340959,1.0
3,-0.411007,-1.242377,1.0
4,-1.658080,-0.262199,NaN


## 4) Ready to code

What do we have so far?

In [88]:
import pandas as pd
b = pd.read_csv('bookings.csv.bz2',sep='^',usecols=['arr_port','pax','year'], nrows=100)
b=b.dropna()

In [89]:
len(b.dropna())

100

In [90]:
b.shape

(100, 3)

In [91]:
b.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


## 4) make action plan
1) get only the bookings from 2013

2) group by arr_port

3) sort 

4) get top 10

#### 4.1) Get only the booking from 2013

In [92]:
b = b[b['year'] == 2013]
b.drop('year',axis=1, inplace=True)
b.head()

,arr_port,pax
0,LHR,-1
1,CLT,1
2,CLT,1
3,SVO,1
4,SVO,1


#### 4.2) group by arr_port

In [93]:
arr_ports=b.groupby('arr_port')

In [100]:
arr_ports.groups

{'ALA     ': [76, 77],
 'ALG     ': [34, 35, 36, 37],
 'AYT     ': [81, 82, 83],
 'BLR     ': [49, 50, 51],
 'BWI     ': [52, 53],
 'CDG     ': [93, 94],
 'CLT     ': [1, 2],
 'CTA     ': [20, 21],
 'CVG     ': [85, 86, 87, 88, 89, 90],
 'DEL     ': [54, 55, 56, 57],
 'DMM     ': [38, 39, 40, 41, 42, 43],
 'FCO     ': [78, 79, 80],
 'HAV     ': [95, 96, 97, 98],
 'LED     ': [44],
 'LGA     ': [5, 6],
 'LHR     ': [0],
 'LIS     ': [45, 46, 47, 48],
 'MNL     ': [59, 60, 61, 62, 63, 64],
 'MVD     ': [84],
 'NRT     ': [99],
 'OTP     ': [91, 92],
 'PVG     ': [30, 31, 32],
 'SGC     ': [33],
 'SIN     ': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
 'SVO     ': [3, 4],
 'TPA     ': [58],
 'TUS     ': [19],
 'ULN     ': [65, 66],
 'YWG     ': [22, 23, 24, 25, 26, 27, 28, 29],
 'ZRH     ': [67, 68, 69, 70, 71, 72, 73, 74, 75]}

In [97]:
arr_ports.get_group('ALA     ')

,arr_port,pax
76,ALA,1
77,ALA,1


In [94]:
for airport, val in arr_ports:
    print "%s" %(airport)
    print val['pax'].sum()

ALA     
2
ALG     
0
AYT     
3
BLR     
1
BWI     
6
CDG     
2
CLT     
2
CTA     
0
CVG     
2
DEL     
0
DMM     
2
FCO     
3
HAV     
0
LED     
1
LGA     
2
LHR     
-1
LIS     
4
MNL     
0
MVD     
2
NRT     
6
OTP     
2
PVG     
6
SGC     
1
SIN     
4
SVO     
2
TPA     
2
TUS     
1
ULN     
2
YWG     
4
ZRH     
-1


In [95]:
for i, (airport, val) in enumerate(arr_ports):
    print "%s" %(airport)
    print val['pax'].sum()
    if (i==10) :break

ALA     
2
ALG     
0
AYT     
3
BLR     
1
BWI     
6
CDG     
2
CLT     
2
CTA     
0
CVG     
2
DEL     
0
DMM     
2


In [102]:
arr_ports.sum().sort_values('pax', ascending=False).head()

,pax
arr_port,
BWI,6
PVG,6
NRT,6
YWG,4
SIN,4


In [104]:
topAirports = b.dropna().groupby(['arr_port']).sum().sort_values('pax', ascending=False).head(10)
#topAirports = b.dropna().groupby(['arr_port']).sum().sort_values('pax', ascending=False)['pax'].head(10)
topAirports

,pax
arr_port,
BWI,6
PVG,6
NRT,6
YWG,4
SIN,4
LIS,4
AYT,3
FCO,3
ALA,2


## 5) Adjust the code to work with Big data

In [110]:
import pandas as pd
path='/home/dsc/Data/challenge'
file_name='bookings.csv.bz2'
b = pd.read_csv(path+'/'+file_name,sep='^',usecols=['arr_port','pax','year'], nrows=1000)
b=b[b['year'] == 2013]\
    [['arr_port','pax']]\
    .dropna()\
    .groupby(['arr_port'])\
    .sum()\
    .sort_values('pax', ascending=False)\
    .head(10)
b

,pax
arr_port,
DCA,50
JFK,14
FLL,13
PVG,13
CGK,12
LGA,12
SVO,12
LAS,11
BOS,11


In [111]:
import pandas as pd
path='/home/dsc/Data/challenge'
file_name='bookings.csv.bz2'
b = pd.read_csv(path+'/'+file_name,sep='^',usecols=['arr_port','pax','year'], nrows=1000)
b=b[b['year'] == 2013]\
    [['arr_port','pax']]\
    .dropna()\
    .groupby(['arr_port'])['pax'].sum().sort_values(ascending=False).head(10)

print type(b)
b

<class 'pandas.core.series.Series'>


arr_port
DCA         50
JFK         14
FLL         13
PVG         13
SVO         12
LGA         12
CGK         12
MIA         11
LAS         11
BOS         11
Name: pax, dtype: int64

#### but... we have to read the whole file... and with nrows we are reading ALWAYS the first N rows...


Option for reading the file:
    1) iterator=true and get_chunk
    2) define chunk_size

#### 1) iterator=true and get_chunk

In [121]:
%%time
b=pd.read_csv('/home/dsc/Data/challenge/bookings.csv.bz2',sep='^',usecols=['arr_port','pax'], iterator=True)
print type(b)
b0=b.get_chunk(6000)
b1=b.get_chunk(3000)
b2=b.get_chunk(1001)
#b2=b.get_chunk(1)

#print b0.head()
#print b1
#print b2


<class 'pandas.io.parsers.TextFileReader'>
CPU times: user 269 ms, sys: 0 ns, total: 269 ms
Wall time: 270 ms


#### 2) define chunk size

In [122]:
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=10)
type (bc)

pandas.io.parsers.TextFileReader

In [120]:
##%%time
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=400)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index().head()
    print "Chank Nummber:%d" % i
    print chunk_airports


Chank Nummber:0
   arr_port  pax
0  ACC         6
1  ADD         2
2  AGP         6
3  ALA         2
4  ALG         0
Chank Nummber:1
   arr_port  pax
0  ALG         0
1  AMD         1
2  AMS         1
3  ANC         0
4  ASU       -10
Chank Nummber:2
   arr_port  pax
0  ALC         4
1  AMS         0
2  ATL         1
3  ATQ         1
4  BCN         1


#### How do we get the all the results of all chunks at one place?


Options:
    1) df.append()
    2) df.concat()
    
    3) list of small DF and then concatenate them in one DF
    4) use dictionaty instead of df

In [142]:

sum_chunks = pd.DataFrame()
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum()
    #chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index().head()
    #chunk_airports=chunk.dropna().groupby(['arr_port']).sum().head() #if we dont reset index arr_port is THe index
    #print chunk_airports
    sum_chunks=sum_chunks.append(chunk_airports)
    #sum_chunks=sum_chunks.append(chunk_airports, ignore_index=True)
sum_chunks


,pax
arr_port,
ACC,6
ADD,2
AGP,6
ALA,2
ALC,4
ALG,0
AMD,1
AMS,1
ANC,0


#### 2) df.concat()

In [140]:
sum_chunks=pd.DataFrame()
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=400)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().head()
    sum_chunks=pd.concat([sum_chunks,chunk_airports])
sum_chunks.reset_index()

,arr_port,pax
0,ACC,6
1,ADD,2
2,AGP,6
3,ALA,2
4,ALG,0
5,ALG,0
6,AMD,1
7,AMS,1
8,ANC,0
9,ASU,-10


 #### Concat and append should produce the same result

In [152]:
#with reset index and group by over column
sum_chunks=pd.DataFrame()
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=4000)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum().reset_index()
    sum_chunks=sum_chunks.append(chunk_airports)
#sum_chunks
topairports=sum_chunks.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index().head(10) 
topairports

,arr_port,pax
0,DCA,50
1,JFK,14
2,FLL,13
3,PVG,13
4,SVO,12
5,LGA,12
6,CGK,12
7,MIA,11
8,LAS,11
9,BOS,11


In [154]:
#without reset_index with group by over index
sum_chunks=pd.DataFrame()
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=400)
for i,chunk in enumerate(bc):
    chunk_airports=chunk.dropna().groupby(['arr_port']).sum()
    sum_chunks=sum_chunks.append(chunk_airports)
#sum_chunks
topairports=sum_chunks.groupby(sum_chunks.index).pax.sum().sort_values(ascending=False).head(10) 
print topairports
topairports.reset_index()

arr_port
DCA         50
JFK         14
FLL         13
PVG         13
SVO         12
LGA         12
CGK         12
MIA         11
LAS         11
BOS         11
Name: pax, dtype: int64


,arr_port,pax
0,DCA,50
1,JFK,14
2,FLL,13
3,PVG,13
4,SVO,12
5,LGA,12
6,CGK,12
7,MIA,11
8,LAS,11
9,BOS,11


#### 3) list of small DF and then concatenate the list in one DF

In [156]:
bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=400)
tops = []
for b in bc:
    small_df=b.dropna().groupby('arr_port').sum().sort_values('pax',ascending=False).reset_index()
    #print list_top[:10]
    tops.append(small_df)
    
top = pd.concat(tops) 
#top

In [160]:
print type(tops)
print type(tops[0])
print type (top)

<type 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [159]:
tops[1].head()

,arr_port,pax
0,DCA,25
1,ORD,8
2,BJV,8
3,DEN,6
4,SJU,6


In [337]:
topairports=top.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index().head(10) 
topairports

,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,SFO,91
5,LAX,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


#### 4) Dictionary

In [164]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'])
#bc = pd.read_csv('bookings.sample.csv.bz2',sep='^',usecols=['arr_port','pax'], chunksize=400)

tops_dict = dict() # moze ovako ili {}
#tops_dict = {}


In [165]:
a=b.dropna().groupby('arr_port').sum().sort_values('pax',ascending=False).reset_index()

In [166]:
a.head()

,arr_port,pax
0,DCA,50
1,JFK,14
2,FLL,13
3,PVG,13
4,CGK,12


In [167]:
a['arr_port'][0]

'DCA     '

In [168]:
d={}
d['Master']='One'

In [171]:
d['Master']

'two'

In [170]:
d['Master']='two'

In [172]:
d['Master']=d['Master']+'one'
d['Master']

'twoone'

In [173]:
d['number']=1
d['number']=d['number']+2
d['number']

3

In [174]:
for b in bc:
    a=b.dropna().groupby('arr_port').sum().sort_values('pax',ascending=False).reset_index()
    for i, row in enumerate(a.iterrows()) :
        airports_name = a.arr_port[i]
        if (not airports_name in tops_dict):
            tops_dict[airports_name]=a.pax[i]
        else:
            tops_dict[airports_name]=tops_dict[airports_name]+a.pax[i]
            

In [175]:
keys=tops_dict.keys()

In [176]:
keys[0]

'AMD     '

In [ ]:
tops_dict.items()

In [59]:
type(tops_dict.items())

list

In [202]:
items=tops_dict.items()
#tops_dict.items()[0][1]

In [203]:
items[:10]

[('AMD     ', 1),
 ('CAG     ', 0),
 ('EWR     ', 8),
 ('DEN     ', 6),
 ('YYZ     ', 2),
 ('FLL     ', 13),
 ('SJU     ', 6),
 ('HER     ', 4),
 ('TUS     ', 2),
 ('LIM     ', 4)]

In [193]:
#import operator
#sorted_list = sorted(tops_dict.items(), key=operator.itemgetter(1), reverse=True)

In [204]:
items.sort(key=lambda x:x[1], reverse=True)
items[:10]

In [206]:
 for num in items[:10]:
        print("    Airport: %s  had %s passengers in 2013" % (num[0], num[1]))

    Airport: DCA       had 50 passengers in 2013
    Airport: JFK       had 14 passengers in 2013
    Airport: FLL       had 13 passengers in 2013
    Airport: PVG       had 13 passengers in 2013
    Airport: CGK       had 12 passengers in 2013
    Airport: SVO       had 12 passengers in 2013
    Airport: LGA       had 12 passengers in 2013
    Airport: MIA       had 11 passengers in 2013
    Airport: LAS       had 11 passengers in 2013
    Airport: BOS       had 11 passengers in 2013


### Another option for sorting the dictonary

In [61]:
tops=zip(tops_dict.keys(), tops_dict.values())

In [62]:
from collections import Counter
Counter(tops_dict).most_common(10)

[('HKG     ', 112),
 ('LGA     ', 95),
 ('ORD     ', 94),
 ('JFK     ', 92),
 ('LAX     ', 91),
 ('SFO     ', 91),
 ('MCO     ', 90),
 ('DCA     ', 82),
 ('DEN     ', 79),
 ('LHR     ', 76)]

## Pre-Bonus Solution

In [217]:
import pandas as pd
path='/home/dsc/Data/challenge'
file_name='bookings.sample.csv.bz2'
#file_name='bookings.csv.bz2'

sum_chunks=pd.DataFrame()

bc = pd.read_csv(path+'/'+file_name,sep='^',usecols=['arr_port','pax','year'], chunksize=300)
for i,chunk in enumerate(bc):
    print i
    chunk_airports=chunk[chunk['year'] == 2013][['arr_port','pax']].dropna().groupby(['arr_port']).sum().reset_index()
    sum_chunks=sum_chunks.append(chunk_airports)
    
topairports=sum_chunks.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index().head(10) 
topairports

0
1
2
3


,arr_port,pax
0,DCA,50
1,JFK,14
2,FLL,13
3,PVG,13
4,SVO,12
5,LGA,12
6,CGK,12
7,MIA,11
8,LAS,11
9,BOS,11


### Now we've got top airports

we have to install geobases
pip install GeoBases

In [218]:
from GeoBases import GeoBase

In [219]:
geoDict = GeoBase(data='ori_por', verbose=False)
#geoDict = GeoBase(data='ori_por')

In [220]:
type(geoDict)

GeoBases.GeoBaseModule.GeoBase

In [221]:
geoDict.keys()[:10]

['MLC',
 'XMZ',
 'XQP@1',
 'JED@1',
 'MFI@1',
 'JGO@1',
 'YAL@1',
 'NDP',
 'GEG@1',
 'NDS']

In [222]:
geoDict.fields

['__key__',
 '__dup__',
 '__par__',
 '__lno__',
 'iata_code',
 'icao_code',
 'faa_code',
 'is_geonames',
 'geoname_id',
 'valid_id',
 'name',
 'asciiname',
 'lat',
 'lng',
 'fclass',
 'fcode',
 'page_rank',
 'date_from',
 'date_until',
 'comment',
 'country_code',
 'cc2',
 'country_name',
 'continent_name',
 'adm1_code',
 'adm1_name_utf',
 'adm1_name_ascii',
 'adm2_code',
 'adm2_name_utf',
 'adm2_name_ascii',
 'adm3_code',
 'adm4_code',
 'population',
 'elevation',
 'gtopo30',
 'timezone',
 'gmt_offset',
 'dst_offset',
 'raw_offset',
 'moddate',
 'city_code',
 'city_name_utf',
 'city_name_ascii',
 'tvl_por_list@raw',
 'tvl_por_list',
 'state_code',
 'location_type@raw',
 'location_type',
 'wiki_link',
 'alt_name_section@raw',
 'alt_name_section',
 '__gar__']

In [223]:
geoDict.get?

In [224]:
geoDict.get('LHR','city_name_utf')

'London'

In [225]:
geoDict.get('LHR')['name']

'London Heathrow Airport'

In [226]:
topairports.arr_port[0]

'DCA     '

### How do we remove white spaces?

In [88]:
topairports.arr_port[0].strip()

'HKG'

### How do we put the name of each airport into the DF?

In [227]:
topairports['AirportName']=geoDict.get(topairports.arr_port[0].strip())['name']

In [228]:
topairports

,arr_port,pax,AirportName
0,DCA,50,Ronald Reagan Washington National Airport
1,JFK,14,Ronald Reagan Washington National Airport
2,FLL,13,Ronald Reagan Washington National Airport
3,PVG,13,Ronald Reagan Washington National Airport
4,SVO,12,Ronald Reagan Washington National Airport
5,LGA,12,Ronald Reagan Washington National Airport
6,CGK,12,Ronald Reagan Washington National Airport
7,MIA,11,Ronald Reagan Washington National Airport
8,LAS,11,Ronald Reagan Washington National Airport
9,BOS,11,Ronald Reagan Washington National Airport


In [231]:
topairports['arr_port'].map(lambda x: geoDict.get(x.strip())['name'])

0            Ronald Reagan Washington National Airport
1                 John F Kennedy International Airport
2      Fort Lauderdale-Hollywood International Airport
3                Shanghai Pudong International Airport
4                   Sheremetyevo International Airport
5                                   La Guardia Airport
6                 Soekarno–Hatta International Airport
7                          Miami International Airport
8                      Mc Carran International Airport
9    General Edward Lawrence Logan International Ai...
Name: arr_port, dtype: object

In [232]:
topairports['Airport Name']=topairports.arr_port.map(lambda x: geoDict.get(x.strip())['name'])
topairports

,arr_port,pax,AirportName,Airport Name
0,DCA,50,Ronald Reagan Washington National Airport,Ronald Reagan Washington National Airport
1,JFK,14,Ronald Reagan Washington National Airport,John F Kennedy International Airport
2,FLL,13,Ronald Reagan Washington National Airport,Fort Lauderdale-Hollywood International Airport
3,PVG,13,Ronald Reagan Washington National Airport,Shanghai Pudong International Airport
4,SVO,12,Ronald Reagan Washington National Airport,Sheremetyevo International Airport
5,LGA,12,Ronald Reagan Washington National Airport,La Guardia Airport
6,CGK,12,Ronald Reagan Washington National Airport,Soekarno–Hatta International Airport
7,MIA,11,Ronald Reagan Washington National Airport,Miami International Airport
8,LAS,11,Ronald Reagan Washington National Airport,Mc Carran International Airport
9,BOS,11,Ronald Reagan Washington National Airport,General Edward Lawrence Logan International Ai...


In [237]:
get_airport_name = lambda x: geoDict.get(x.strip())['name']
topairports.arr_port.map(get_airport_name)

0            Ronald Reagan Washington National Airport
1                 John F Kennedy International Airport
2      Fort Lauderdale-Hollywood International Airport
3                Shanghai Pudong International Airport
4                   Sheremetyevo International Airport
5                                   La Guardia Airport
6                 Soekarno–Hatta International Airport
7                          Miami International Airport
8                      Mc Carran International Airport
9    General Edward Lawrence Logan International Ai...
Name: arr_port, dtype: object

In [238]:
topairports.drop('AirportName', axis=1, inplace=True)
topairports

ValueError: labels ['AirportName'] not contained in axis

In [240]:
topairports['AirportName']=geoDict.get(topairports.arr_port[0].strip())['name']
topairports

,arr_port,pax,Airport Name,AirportName
0,DCA,50,Ronald Reagan Washington National Airport,Ronald Reagan Washington National Airport
1,JFK,14,John F Kennedy International Airport,Ronald Reagan Washington National Airport
2,FLL,13,Fort Lauderdale-Hollywood International Airport,Ronald Reagan Washington National Airport
3,PVG,13,Shanghai Pudong International Airport,Ronald Reagan Washington National Airport
4,SVO,12,Sheremetyevo International Airport,Ronald Reagan Washington National Airport
5,LGA,12,La Guardia Airport,Ronald Reagan Washington National Airport
6,CGK,12,Soekarno–Hatta International Airport,Ronald Reagan Washington National Airport
7,MIA,11,Miami International Airport,Ronald Reagan Washington National Airport
8,LAS,11,Mc Carran International Airport,Ronald Reagan Washington National Airport
9,BOS,11,General Edward Lawrence Logan International Ai...,Ronald Reagan Washington National Airport


In [241]:
del topairports['AirportName']
topairports

,arr_port,pax,Airport Name
0,DCA,50,Ronald Reagan Washington National Airport
1,JFK,14,John F Kennedy International Airport
2,FLL,13,Fort Lauderdale-Hollywood International Airport
3,PVG,13,Shanghai Pudong International Airport
4,SVO,12,Sheremetyevo International Airport
5,LGA,12,La Guardia Airport
6,CGK,12,Soekarno–Hatta International Airport
7,MIA,11,Miami International Airport
8,LAS,11,Mc Carran International Airport
9,BOS,11,General Edward Lawrence Logan International Ai...


In [96]:
topairports

,arr_port,pax,Airport Name
0,HKG,112,Hong Kong International Airport
1,LGA,95,La Guardia Airport
2,ORD,94,Chicago O'Hare International Airport
3,JFK,92,John F Kennedy International Airport
4,SFO,91,San Francisco International Airport
5,LAX,91,Los Angeles International Airport
6,MCO,90,Orlando International Airport
7,DCA,82,Ronald Reagan Washington National Airport
8,DEN,79,Denver International Airport
9,LHR,76,London Heathrow Airport


In [242]:
topairports.drop(topairports.index[[8,3]], axis=0)


,arr_port,pax,Airport Name
0,DCA,50,Ronald Reagan Washington National Airport
1,JFK,14,John F Kennedy International Airport
2,FLL,13,Fort Lauderdale-Hollywood International Airport
4,SVO,12,Sheremetyevo International Airport
5,LGA,12,La Guardia Airport
6,CGK,12,Soekarno–Hatta International Airport
7,MIA,11,Miami International Airport
9,BOS,11,General Edward Lawrence Logan International Ai...


In [243]:
topairports.ix[8]

arr_port                               LAS     
pax                                          11
Airport Name    Mc Carran International Airport
Name: 8, dtype: object

In [248]:
topairports[topairports.pax>12].index

Int64Index([0, 1, 2, 3], dtype='int64')

In [249]:
topairports.drop(topairports[topairports.pax>90].index, axis=0)

,arr_port,pax,Airport Name
0,DCA,50,Ronald Reagan Washington National Airport
1,JFK,14,John F Kennedy International Airport
2,FLL,13,Fort Lauderdale-Hollywood International Airport
3,PVG,13,Shanghai Pudong International Airport
4,SVO,12,Sheremetyevo International Airport
5,LGA,12,La Guardia Airport
6,CGK,12,Soekarno–Hatta International Airport
7,MIA,11,Miami International Airport
8,LAS,11,Mc Carran International Airport
9,BOS,11,General Edward Lawrence Logan International Ai...


## Final Solution

In [3]:
%%time
import pandas as pd
path='/home/dsc/Data/challenge'
file_name='bookings.csv.bz2'
#file_name='bookings.csv.bz2'

sum_chunks=pd.DataFrame()

bc = pd.read_csv(path+'/'+file_name,sep='^',usecols=['arr_port','pax','year'], chunksize=1000000)
bc = pd.read_csv(path+'/'+file_name,sep='^',usecols=['arr_port','pax','year'], chunksize=3000) #for sample file
for i,chunk in enumerate(bc):
    print i
    chunk_airports=chunk[chunk['year'] == 2013][['arr_port','pax']].dropna().groupby(['arr_port']).sum().reset_index()
    sum_chunks=sum_chunks.append(chunk_airports)
    
#topairports=sum_chunks.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index().head(10) 

#we are going to need this for the web service so we'll put it without top10
topairports=sum_chunks.groupby('arr_port').pax.sum().sort_values(ascending=False).reset_index() 

from GeoBases import GeoBase
geoDict = GeoBase(data='ori_por', verbose=False)
topairports['Airport Name']=topairports.arr_port.map(lambda x: geoDict.get(x.strip())['name'])

topairports.head()

0
1
2
3
4
5
6
7
8
9
10
CPU times: user 3min 34s, sys: 1.99 s, total: 3min 36s
Wall time: 3min 37s


In [9]:
topairports.to_csv('top_airports.csv',sep='^', index=False)
!pwd

/home/dsc/Data/DS_Master_A1_Chall


In [10]:
ls -l

total 91672
drwxrwxr-x. 2 dsc dsc     4096 May  5 11:57 backup/
-rw-rw-r--. 1 dsc dsc 75836317 May  6 07:08 bookings_wn_dup.csv.bz2
-rw-rw-r--. 1 dsc dsc     5294 May  5 11:57 ch_01-Empty.ipynb
-rw-rw-r--. 1 dsc dsc    14580 May  5 11:57 ch_01.ipynb
-rw-rw-r--. 1 dsc dsc    12930 May  5 23:55 ch_02-Empty.ipynb
-rw-rw-r--. 1 dsc dsc   237922 May  6 14:09 ch_02.ipynb
-rw-rw-r--. 1 dsc dsc    10832 May  5 23:58 ch_03-Empty.ipynb
-rw-rw-r--. 1 dsc dsc   332172 May  6 12:27 ch_03.ipynb
-rw-rw-r--. 1 dsc dsc     8019 May  6 12:19 ch_04-Empty.ipynb
-rw-rw-r--. 1 dsc dsc   119881 May  6 12:36 ch_04.ipynb
-rw-rw-r--. 1 dsc dsc     7194 May  5 11:57 ch_05_b.ipynb
-rw-rw-r--. 1 dsc dsc     9844 May  6 12:36 ch_05.ipynb
drwxrwxr-x. 2 dsc dsc     4096 May  5 11:57 DaniPythonClass/
drwxrwxr-x. 3 dsc dsc     4096 May  5 11:57 DaniPythonClassWorking/
drwxrwxr-x. 2 dsc dsc     4096 May  5 11:57 IsraChall/
-rw-rw-r--. 1 dsc dsc    19978 May  6 11:45 playground.ipynb
-rw-rw-r--. 1 dsc dsc       20 May  5

In [11]:
topairports.head()

,arr_port,pax,Airport Name
0,LHR,88809,London Heathrow Airport
1,MCO,70930,Orlando International Airport
2,LAX,70530,Los Angeles International Airport
3,LAS,69630,Mc Carran International Airport
4,JFK,66270,John F Kennedy International Airport


## Drop Duplicates()

In [127]:
df=df.append(df.ix[2]).reset_index()
df

,index,one,two,three,four,five
0,a,-1.652355,-0.033970,-0.456528,bar,NaN
1,c,-2.767755,1.406146,-0.771437,bar,NaN
2,e,0.402114,0.759691,0.090970,bar,1
3,f,-0.774120,-0.111994,-0.265381,bar,1
4,h,-0.627316,2.166776,-0.553747,bar,NaN
5,e,0.402114,0.759691,0.090970,bar,1
6,e,0.402114,0.759691,0.090970,bar,1


In [129]:
df.drop_duplicates()

,index,one,two,three,four,five
0,a,-1.652355,-0.033970,-0.456528,bar,NaN
1,c,-2.767755,1.406146,-0.771437,bar,NaN
2,e,0.402114,0.759691,0.090970,bar,1
3,f,-0.774120,-0.111994,-0.265381,bar,1
4,h,-0.627316,2.166776,-0.553747,bar,NaN


In [130]:
df.dropna().drop_duplicates()

,index,one,two,three,four,five
2,e,0.402114,0.759691,0.090970,bar,1
3,f,-0.774120,-0.111994,-0.265381,bar,1


### Compare two DF

In [250]:
b0=pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)
b1=pd.read_csv('bookings.csv.bz2',sep='^', nrows=1000)
b2=pd.read_csv('bookings.csv.bz2',sep='^', nrows=100)


In [251]:
from pandas.util.testing import assert_frame_equal
assert_frame_equal(b0, b1)

In [252]:
assert_frame_equal(b0, b2)



AssertionError: DataFrame are different

DataFrame shape (number of rows) are different
[left]:  1000, RangeIndex(start=0, stop=1000, step=1)
[right]: 100, RangeIndex(start=0, stop=100, step=1)

In [253]:
def compare_df (a, b):
    try:
        assert_frame_equal(a, b)
        return True
    except:  # appeantly AssertionError doesn't catch all
        return False

In [254]:
compare_df(b0,b1)

True

In [255]:
compare_df(b0,b2)

False